In [1]:
from new_basis_llp_qaoa.statevector_sim import CYPStateVector 
from new_basis_llp_qaoa.statevector_sim.cyp_statevector_sim import CYPStatevectorSim
import numpy as np

In [2]:

def generate_random_dataset(M, N, strength=0.2):
    As = np.zeros((N,N))
    for i in range(N):
        for j in range(i):
            x = np.random.random()
            while x > strength:
                x = np.random.random()
            As[i][j] = x
            As[j][i] = x

    Qs = np.zeros((N,M))
    expected_returns = np.linspace(1/M, 1, M)[:: -1]*(1/M)
    Vs = As.sum(axis=1)
    for i in range(N):
        np.random.shuffle(expected_returns)
        Qs[i, :] = Vs[i] * expected_returns
    return As, Qs


In [3]:
import numpy as np
ASList = np.load("../../resource/sample_matrices/AS-Copy1.npy")
QSList = np.load("../../resource/sample_matrices/QS-Copy1.npy")
AS = ASList[0]
QS = QSList[0]

N, M = QS.shape


In [4]:
N_OP = np.array([[0,0], [0,1]])
PAULI_X = np.array([[0,1], [1,0]])
XNX_OP = PAULI_X @ N_OP @ PAULI_X
PAULI_I = np.identity(2)

In [5]:
import numpy as np

def binary_1(j):
    bit_string = bin(j)[2:][::-1]
    indexed_bits = enumerate(bit_string)
    return set(index for index, bit in indexed_bits if bit == '1')

def binary_0(j):
    bit_string = bin(j)[2:][::-1]
    indexed_bits = enumerate(bit_string)
    return set(index for index, bit in indexed_bits if bit == '0')

def qubits_number_for_number_of_actions(action):
    return int(np.ceil(np.log2(action)))


def get_qubits_number(loanee, action):
    return qubits_number_for_number_of_actions(action) * loanee

def nu(action_num, max_number_of_qubits):
    res = N_OP if 0 in binary_1(action_num) else XNX_OP

    for i in range(1, max_number_of_qubits):
        if i in binary_1(action_num):
            res = np.kron(N_OP, res)
        else:
            res = np.kron(XNX_OP, res)
    return res
max_n = qubits_number_for_number_of_actions(3)
nu8 = nu(0, max_n)
nu8, max_n

(array([[1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]),
 2)

In [6]:
def nu_ij(i, j, max_n, loanees):
    before = i*max_n
    after = (loanees - i - 1) * max_n
    before = np.identity(2**before)
    after = np.identity(2**after)
    res = np.kron(nu(j, max_n), before)
    res = np.kron(after, res)
    return res



In [67]:
np.random.seed(42)
As, Qs = generate_random_dataset(3,2)
As, Qs.shape

(array([[0.        , 0.15601864],
        [0.15601864, 0.        ]]),
 (2, 3))

In [8]:
#def profit_term()
max_n = qubits_number_for_number_of_actions(3)
n = get_qubits_number(loanee=2, action=3)
profit_term = np.zeros((2**n,2**n))
for i in range(2):
    for j in range(3):
        h_ij = Qs[i, j]
        profit_term += h_ij * nu_ij(i,j, max_n, loanees=2)


In [41]:
def find_closest(pin, targets):
    score = 1000
    res = None
    for i, target in enumerate(targets):
        new_score = np.abs(pin - target)
        if new_score < score:
            score = new_score
            res = target, i
    return res

In [10]:
welfare_term = np.zeros((2**n,2**n))
for i in range(2):
    for i_prime in range(i+1, 2):
        first_factor = np.identity(2**n) - nu_ij(i,0, max_n, loanees=2)
        second_factor = np.identity(2**n) - nu_ij(i_prime,0, max_n, loanees=2)
        welfare_term += As[i, i_prime] * (first_factor @ second_factor)


In [11]:
new_HA = -(1-1/20) * profit_term - 1/20 * welfare_term
new_HA.diagonal()

array([-0.08234317, -0.09881181, -0.06587454, -0.0494059 , -0.06587454,
       -0.0901441 , -0.05720683, -0.0407382 , -0.0494059 , -0.07367547,
       -0.0407382 , -0.02426957, -0.03293727, -0.05720683, -0.02426957,
       -0.00780093])

In [17]:
for term in new_HA.diagonal():
    print(term)
    print(find_closest(term, old_HA.diagonal()))
    continue


-0.08234317134461927
-0.08234317134461927
-0.09881180561354312
-0.09881180561354314
-0.0658745370756954
-0.0658745370756954
-0.04940590280677156
-0.04940590280677157
-0.06587453707569542
-0.06587453707569542
-0.0901441033667411
-0.0901441033667411
-0.057206834828893396
-0.057206834828893396
-0.04073820055996954
-0.04073820055996954
-0.04940590280677157
-0.04940590280677157
-0.07367546909781723
-0.07367546909781723
-0.04073820055996953
-0.04073820055996953
-0.024269566291045677
-0.024269566291045677
-0.03293726853784771
-0.0329372685378477
-0.05720683482889339
-0.05720683482889339
-0.024269566291045677
-0.024269566291045677
-0.007800932022121826
-0.007800932022121826


In [12]:
old = CYPStatevectorSim(Qs=Qs, As= As, J=1, Epsilon=1/20, Parameters=[1],InitialState="ones")
old_HA = old.HA

In [64]:
for i in range(2**n):
    state = np.zeros(2**n)
    state[i] = 1
    h = new_HA.diagonal() @ state
    indicies = np.where(np.isclose(h, old_HA.diagonal()))[0]
    print(bin(i)[2:])
    print("-"*5)
    for index in indicies:
        print(bin(index)[2:])
    print("*"*5)




0
-----
110
1101
100011
100100
101010
110000
*****
1
-----
111
1110
10100
100101
101011
101100
110001
111000
*****
10
-----
101
1100
100010
101001
*****
11
-----
100
100001
101000
*****
100
-----
101
1100
100010
101001
*****
101
-----
10010
11001
*****
110
-----
11
1010
10000
*****
111
-----
10
1001
*****
1000
-----
100
100001
101000
*****
1001
-----
1011
10001
11000
*****
1010
-----
10
1001
*****
1011
-----
1
1000
*****
1100
-----
100000
*****
1101
-----
11
1010
10000
*****
1110
-----
1
1000
*****
1111
-----
0
*****


In [65]:
Qs

array([[0.03467081, 0.05200621, 0.0173354 ],
       [0.05200621, 0.03467081, 0.0173354 ]])

In [ ]:
old_HA.diagonal()[]

In [181]:
max_n = qubits_number_for_number_of_actions(3)
n = get_qubits_number(loanee=2, action=3)
i = np.random.randint(5)
j = np.random.randint(3)
nu_ij(i,j, max_n, loanees=2)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [42]:
np.kron(PAULI_I, np.kron(PAULI_I, PAULI_I))
np.kron(np.identity(1), np.identity(2))


array([[1., 0.],
       [0., 1.]])

In [17]:
la = np.kron(PAULI_X, PAULI_X)
la = np.kron(PAULI_X, la)
la = np.kron(PAULI_X, la)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,